# Test-benchmark using fake data

In [4]:
%load_ext autoreload
%autoreload 2

import sys
import os
import warnings
warnings.filterwarnings("ignore")

# to remplace with pip install abench and uqmodels
path = '../'
sys.path.insert(1, path)
sys.path.insert(1, path+'src')

import abench as abench
import uqmodels as uqmodels
import abench.store as A_store
import matplotlib.pyplot as plt
import numpy as np
import pickle

data_folder = path+'data/Public_dataset/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Preprocessing

# Formalisation modèle

In [3]:
#Data_processor for preprocessed data stored in a dict
from abench.benchmark import dataset_generator_from_array,splitter,TimeSeriesSplit,KFold,splitter
from abench.store import store_data_generator
from src.Benchmark_UQ import Encapsulated_model_UQ,TimeSeries_from_dict
from src.attack_function import ctx_seq_attack_UC_AL, ctx_seq_attack, test_attack

#Choice of public dataset
num_dataset = 1
data_folder = path+'data/Public_dataset/'
n_repet = 1

if(num_dataset==1):
    storing =path+'results/Benchmark_public/Benchmark_public_dataset_kin8nm'
    dataset = pd.read_csv(data_folder+"dataset_2175_kin8nm.csv")
    print(dataset.shape)
    dataset.head()
    X = dataset[['theta1','theta2','theta3','theta4','theta5','theta6','theta7','theta8']].values
    y = dataset['y'].values.reshape(-1,1)
    X_shape,y_shape=X.shape[-1],y.shape[-1]
    split=KFold(5)
    name_split = ['cv_1','cv_2','cv_3','cv_4','cv_5']
    dataset_generator = dataset_generator_from_array(X,y,None,None,
                                                     sk_split=split,
                                                     remove_from_train=None,
                                                     attack_name="",
                                                     cv_list_name=name_split)
    if not os.path.exists(storing):
        print('Generation of data-generator at '+storing)
        store_data_generator(storing,data_generator)
    else:
        print(storing+' already exists')
    
elif(num_dataset==2):
    storing =path+'results/Benchmark_public/Benchmark_public_dataset_Folds5x2_pp'
    dataset = pd.read_csv(data_folder+"Folds5x2_pp.csv")
    X = dataset[['AT','V','AP','RH']].values
    y = dataset['PE'].values.reshape(-1,1)
    X_shape,y_shape=X.shape[-1],y.shape[-1]
    split=KFold(5)
    name_split = ['cv_1','cv_2','cv_3','cv_4','cv_5']
    if not os.path.exists(storing):
        print('Generation of data-generator at '+storing)
        store_data_generator(storing,data_generator)
    else:
        print(storing+' already exists')

elif(num_dataset==3):
    storing =path+'results/Benchmark_public/Benchmark_public_dataset_CASP_protein'
    dataset = pd.read_csv(data_folder+"CASP_protein.csv")
    X = dataset.iloc[:,1:].values
    y = dataset.iloc[:,:1].values.reshape(-1,1)
    X_shape,y_shape=X.shape[-1],y.shape[-1]
    name_split = ['cv_1','cv_2','cv_3','cv_4','cv_5']
    split=KFold(5)
    if not os.path.exists(storing):
        print('Generation of data-generator at '+storing)
        store_data_generator(storing,data_generator)
    else:
        print(storing+' already exists')
    
elif(num_dataset==4):
    print('Warning : big dataset may take some minutes')
    storing =path+'results/Benchmark_public/Benchmark_public_dataset_year_prediction'
    X_shape,y_shape=90,-1
    name_split = ['cv_1']
    if not os.path.exists(storing):
        print('Warning : big dataset, may take some minutes')
        dataset = pd.read_csv(data_folder+"year_prediction.csv")
        X=dataset[0][0]
        y=dataset[0][1]
        split = sk_split=splitter(np.arange(515345)>463715)   #train: first 463,715 examples test: last 51,630 examples
        dataset_generator = dataset_generator_from_array(X,y,None,None,
                                             sk_split=split,
                                             repetition=n_repet,
                                             remove_from_train=None,
                                             attack_name="",cv_list_name=name_split)
        print('Generation of data-generator at '+storing)
        store_data_generator(storing,data_generator)
    else:
        print(storing+' already exists')

2023-05-15 12:14:39.427254: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(8192, 9)
results/Benchmark_public/Benchmark_public_dataset_kin8nm already exists


In [15]:
from copy import deepcopy
from uqmodels.common.predictors import PredictorRF_UQ_distangle
from sklearn.ensemble import RandomForestRegressor

print(num_dataset)
#Tunning process : 
if(False):
    rf_config = {'n_estimators': [100,200,300],
     'min_impurity_decrease': [0.00001,0.0001,0.001, 0.01],
     'ccp_alpha': [0.00000,0.00001,0.0001, 0.001],
     'max_features': [0.5, 0.7, 0.9, 0.99],
     'max_samples': [0.5, 0.7, 0.9, 0.99],
     'max_depth': [6, 9, 12, 15, 20],
     'min_samples_split': [2,4,6, 8, 10, 15],
     'min_samples_leaf': [2,4,6,8,10,15]}
    RF = PredictorRF_UQ_distangle()
    X_,y_,split_,_,_,_ = dataset_generator[0]
    print(len(X_))
    train = split_==1
    RF_tuning(X_[train], y_[train,0], n_esti=200, folds=3, params=rf_config)
    estimator = RF.estimator_mean

elif(num_dataset==1):
    estimator = RandomForestRegressor(ccp_alpha=1e-05, max_depth=15, max_features=0.5,
                      max_samples=0.99, min_impurity_decrease=1e-05,
                      min_samples_leaf=2, min_samples_split=6)
    
elif(num_dataset==2):
    estimator = RandomForestRegressor(ccp_alpha=1e-05, max_depth=20, max_features=0.7,
                          max_samples=0.99, min_impurity_decrease=0.0001,
                          min_samples_leaf=2, min_samples_split=8)
    
elif(num_dataset==3):
    estimator = RandomForestRegressor(ccp_alpha=1e-05, max_depth=20, max_features=0.7,
                          max_samples=0.99, min_impurity_decrease=0.0001,
                          min_samples_leaf=2, min_samples_split=8)
    
elif(num_dataset==4):
    estimator = RandomForestRegressor(ccp_alpha=0.0001, max_depth=15, max_features=0.7,
                      max_samples=0.9, min_impurity_decrease=0.0001,
                      min_samples_leaf=10, min_samples_split=8,
                      n_estimators=300)

RF_dUQ_parameters = {'estimator':estimator,
                 'pretuned':False,
                 'mode':'sigma',
                 'use_biais':True,
                 'var_min':0.00001,
                 'rescale':True}

# MLP - BNN
from uqmodels.common.basic_NN import build_MSE_loss,build_BNN_loss,mlp,default_callbacks

callbacks = default_callbacks(min_delta=0.001,earlystop_patience=60,reducelr_patience=30,reducelr_factor=0.4,reduce_lr_min_lr=5e-06,verbose=0,)
step = 500
type_var = 'MC_Dropout'
# PNN_MCDP
if(num_dataset==1):
    model_param={'dim_in':X_shape,'dim_out':y_shape, 'layers_size':[200,300,200],
                 'regularizer_W':(0.0002,0.0002),'name':'','dp':0.10,'type_var':type_var,
                 'logvar_min':np.log(0.00001)}
    
    training_param = {'epochs':[step,step,step],'b_s':[128,32,64],'l_r':[0.001,0.0005,0.0001],
                  'sample_w':None,'list_loss':[build_BNN_loss],
                  'metrics':[build_MSE_loss(2,True),build_BNN_loss(0.9,True)],
                  'param_loss':[1.15],'callbacks':callbacks}

if(num_dataset==2):
    model_param={'dim_in':X_shape,'dim_out':y_shape, 'layers_size':[200,300,200],
                 'regularizer_W':(0.0002,0.0002),'name':'','dp':0.10,'type_var':type_var,
                 'logvar_min':np.log(0.00001)}
        
    training_param = {'epochs':[step,step,step],'b_s':[128,32,64],'l_r':[0.001,0.0005,0.0001],
                  'sample_w':None,'list_loss':[build_BNN_loss],
                  'metrics':[build_MSE_loss(2,True),build_BNN_loss(0.9,True)],
                  'param_loss':[1.15],'callbacks':callbacks}
                      
if(num_dataset==3):
    model_param={'dim_in':X_shape,'dim_out':y_shape, 'layers_size':[200,300,200],
                 'regularizer_W':(0.00008,0.00008),'name':'','dp':0.13,'type_var':type_var,
                 'logvar_min':np.log(0.00001)}
        
    training_param = {'epochs':[step,step,step],'b_s':[128,32,64],'l_r':[0.001,0.0005,0.0001],
                  'sample_w':None,'list_loss':[build_BNN_loss],
                  'metrics':[build_MSE_loss(2,True),build_BNN_loss(0.9,True)],
                  'param_loss':[1.15],'callbacks':callbacks}
    
if(num_dataset==4):
    model_param={'dim_in':X_shape,'dim_out':y_shape, 'layers_size':[200,300,300,200],
                'regularizer_W':(0.0005,0.0005),'name':'','dp':0.22,'type_var':type_var,
                 'logvar_min':np.log(0.00001)}
    
    training_param = {'epochs':[step,step,step],'b_s':[256,64,126],'l_r':[0.001,0.0005,0.0001],
                  'sample_w':None,'list_loss':[build_BNN_loss],
                  'metrics':[build_MSE_loss(2,True),build_BNN_loss(0.9,True)],
                  'param_loss':[1.1],'callbacks':callbacks}


PNN_MCDP_parameters = {'rescale':True,
                     'model_initializer':mlp,
                     'model_parameters':model_param,
                     'training_parameters':training_param,
                     'type_var':type_var}

# EDL 
from uqmodels.common.basic_NN import build_EDL_loss,build_BNN_loss,mlp
callbacks = default_callbacks(min_delta=0.005,earlystop_patience=60,reducelr_patience=30,reducelr_factor=0.4,reduce_lr_min_lr=5e-06,verbose=0)
step = 500
type_var = 'EDL'
if(num_dataset==1):
    model_param={'dim_in':X_shape,'dim_out':y_shape, 'layers_size':[200,300,200],
                 'regularizer_W':(0.0008,0.0008),'name':'','dp':0.03,'type_var':type_var,
                 'logvar_min':np.log(0.00005)}
    
    training_param = {'epochs':[step,step,step],'b_s':[128,32,64],
                      'l_r':[0.001,0.0005,0.0001],'sample_w':None,'list_loss':[build_EDL_loss],
                      'metrics':[build_MSE_loss(4,True),build_BNN_loss(0.9,True,type_var)],
                      'callbacks':callbacks,'param_loss':[2e-1]}
if(num_dataset==2):
    model_param={'dim_in':X_shape,'dim_out':y_shape, 'layers_size':[200,300,200],
                 'regularizer_W':(0.00005,0.00005),'name':'','dp':0.02,'type_var':type_var,
                 'logvar_min':np.log(0.00005)}
    
    training_param = {'epochs':[step,step,step],'b_s':[128,32,64],
                      'l_r':[0.001,0.0005,0.0001],'sample_w':None,'list_loss':[build_EDL_loss],
                      'metrics':[build_MSE_loss(4,True),build_BNN_loss(0.9,True,type_var)],
                      'callbacks':callbacks,'param_loss':[10e-3]}
    
if(num_dataset==3):
    model_param={'dim_in':X_shape,'dim_out':y_shape, 'layers_size':[200,300,200],
             'regularizer_W':(0.00005,0.00005),'name':'','dp':0.02,'type_var':type_var,
             'logvar_min':np.log(0.00005)}
    
    training_param = {'epochs':[step,step,step],'b_s':[128,32,64],
                      'l_r':[0.001,0.0005,0.0001],'sample_w':None,'list_loss':[build_EDL_loss],
                      'metrics':[build_MSE_loss(4,True),build_BNN_loss(0.9,True,type_var)],
                      'callbacks':callbacks,'param_loss':[5e-4]}
    
if(num_dataset==4):    
    model_param={'dim_in':X_shape,'dim_out':y_shape, 'layers_size':[200,300,300,200],
             'regularizer_W':(0.0004,0.0004),'name':'','dp':0.02,'type_var':type_var,
             'logvar_min':np.log(0.00005)}
    
    training_param = {'epochs':[step,step,step],'b_s':[256,64,128],
                      'l_r':[0.001,0.0005,0.0001],'sample_w':None,'list_loss':[build_EDL_loss],
                      'metrics':[build_MSE_loss(4,True),build_BNN_loss(0.9,True,type_var)],
                      'callbacks':callbacks,'param_loss':[1e-5]}
    

EDL_parameters = {'rescale':True,
                  'model_initializer':mlp,
                  'model_parameters':model_param.copy(),
                  'training_parameters':training_param.copy(),
                  'type_var': 'EDL'}

#PNN_DE
type_var = 'Deep_ensemble'
if(num_dataset==1):
    model_param={'dim_in':X_shape,'dim_out':y_shape, 'layers_size':[200,300,200],
                 'n_ech':6,'k_fold':None,"snapshot":False,'data_drop':0.1,'train_ratio':0.95,'ddof':1,
                 'regularizer_W':(0.002,0.002),'name':'','dp':0.02,'type_var':type_var,
                 'logvar_min':np.log(0.00001)}

    training_param = {'epochs':[step,step],'b_s':[200,100],'l_r':[0.001,0.0005],'sample_w':None,
        'list_loss':[build_BNN_loss],'metrics':[build_MSE_loss(2,True),build_BNN_loss(0.9,True)],
        'param_loss':[0.85],'callbacks':callbacks}
    
if(num_dataset==2):
    model_param={'dim_in':X_shape,'dim_out':y_shape, 'layers_size':[200,300,200],
             'n_ech':4,'k_fold':None,"snapshot":False,'data_drop':0,'train_ratio':0.95,'ddof':1,
             'regularizer_W':(0.00003,0.00003),'name':'','dp':0.02,'type_var':type_var,
             'logvar_min':np.log(0.00001)}

    training_param = {'epochs':[step,step,step],'b_s':[128,32,64],'l_r':[0.001,0.0005,0.0005],'sample_w':None,
        'list_loss':[build_BNN_loss],'metrics':[build_MSE_loss(2,True),build_BNN_loss(0.9,True)],
        'param_loss':[1.5],'callbacks':callbacks}
    
if(num_dataset==3):
    model_param={'dim_in':X_shape,'dim_out':y_shape, 'layers_size':[200,300,200],
         'n_ech':4,'k_fold':None,"snapshot":False,'data_drop':0,'train_ratio':0.95,'ddof':1,
         'regularizer_W':(0.00003,0.00003),'name':'','dp':0.02,'type_var':type_var,
         'logvar_min':np.log(0.00001)}

    training_param = {'epochs':[step,step,step],'b_s':[128,32,64],'l_r':[0.001,0.0005,0.0005],'sample_w':None,
        'list_loss':[build_BNN_loss],'metrics':[build_MSE_loss(2,True),build_BNN_loss(0.9,True)],
        'param_loss':[1.5],'callbacks':callbacks}
    
if(num_dataset==4):
    model_param={'dim_in':X_shape,'dim_out':y_shape, 'layers_size':[200,300,300,200],
         'n_ech':4,'k_fold':None,"snapshot":False,'data_drop':0,'train_ratio':0.95,'ddof':1,
         'regularizer_W':(0.0003,0.0003),'name':'','dp':0.02,'type_var':type_var,
         'logvar_min':np.log(0.00001)}

    training_param = {'epochs':[step,step,step],'b_s':[256,64,128],'l_r':[0.001,0.0005,0.0005],
                      'sample_w':None,'list_loss':[build_BNN_loss],
                      'metrics':[build_MSE_loss(2,True),build_BNN_loss(0.9,True)],
                      'param_loss':[1.5],'callbacks':callbacks}


PNN_DE_parameters = {'rescale':True,
                              'model_initializer':mlp,
                              'model_parameters':model_param.copy(),
                              'training_parameters':training_param.copy(),
                              'type_var':type_var}

1


# Formalisation tache

In [16]:
from sklearn.ensemble import RandomForestRegressor
from uqmodels.common.predictors import PredictorRF_UQ_distangle
from uqmodels.common.neural_network_UQ import NN_var
random = np.random.randint(10000000)
# Models initializer and parameters stored.
dict_Predictor={}

dict_Predictor['RF_dUQ'] = {'subpart':PredictorRF_UQ_distangle,'parameters':RF_dUQ_parameters}
dict_Predictor['PNN_MCDP'] = {'subpart':NN_var,'parameters':PNN_MCDP_parameters}
dict_Predictor['PNN_DE'] = {'subpart':NN_var,'parameters':PNN_DE_parameters}      
dict_Predictor['EDL'] = {'subpart':NN_var,'parameters':EDL_parameters}

exp_design=[]
# Experiment plan specification
exp_design.append([{'name':'PNN_MCDP_test','model':'PNN_MCDP'},
                   {'name':'PNN_DE_test','model':'PNN_DE'},
                   {'name':'EDL_test','model':'EDL'}])

# Model wrapper :
from src.Benchmark_UQ import Encapsulated_model_UQ
# Wrapper configuration provided to abench (benchmark tool)
dict_exp={'encapsulated_model': Encapsulated_model_UQ,
          'tuning_scheme' : {'model':None},
          'model' : dict_Predictor,
          'exp_design':exp_design}

In [ ]:
# Metrics definition to encapsulate in Generic_metric
cv_list=['cv_1']
print(storing,cv_list)

from src.Benchmark_UQ import rmse,average_coverage,sharpness,Gaussian_NLL
from abench.benchmark import Generic_metric

list_metrics=[Generic_metric(rmse,'Rmse_all', mask=None,list_ctx_constraint=None,reduce=True),
              Generic_metric(Gaussian_NLL,"NLL", mask=None,list_ctx_constraint=None,reduce=True),
              Generic_metric(average_coverage,"Cov_ALL", mask=None,list_ctx_constraint=None,reduce=True,type_var="all"),
              Generic_metric(sharpness,"Aw_E", mask=None,list_ctx_constraint=None,reduce=True,type_var="epistemic"),
              Generic_metric(sharpness,"Aw_A", mask=None,list_ctx_constraint=None,reduce=True,type_var="aleatoric")]

obj_param = {'alpha':0.05}
tuning_kwarg = {}

#Start from empty dict
from abench.benchmark import benchmark
benchmark(storing,None,dict_exp,obj_param,list_metrics,tuning_kwarg=tuning_kwarg,verbose=0,cv_list=cv_list)

results/Benchmark_public/Benchmark_public_dataset_kin8nm ['cv_1']
n_experiments : 0
PNN_MCDP_test
load existant data_set
start cv_1
start_fit
256/256 [==============================] - 1s 3ms/step
cv_1 len_train 6553
[(['Rmse_all'], [0.037960100870044214, 0.06898083185931006]), (['NLL'], [-1.8564080708094999, -1.1753754934756029]), (['Cov_ALL'], [0.9851976194109568, 0.849908480780964]), (['Aw_E'], [0.12353160675497799, 0.12654064114351518]), (['Aw_A'], [0.16266245625043438, 0.16310971909415745])]
cv_1 len_train 6553
PNN_DE_test
start cv_1
start_fit
0.8544178238974516 6


# Performance results on the 4 publics dataset

In [5]:
from abench.benchmark import evaluate, Generic_metric
from src.Benchmark_UQ import rmse,average_coverage,sharpness,Gaussian_NLL

storing =path+'results/Benchmark_public/Benchmark_public_dataset_kin8nm'
cv_list =  ['cv_1','cv_2','cv_3','cv_4','cv_5']
list_name = ['RF_dUQ','PNN_MCDP','PNN_DE','EDL']
list_ctx_constraint=None
list_metrics = [Generic_metric(rmse,'Rmse_all', mask=None,list_ctx_constraint=list_ctx_constraint,reduce=True),
                Generic_metric(Gaussian_NLL,"NLL", mask=None,list_ctx_constraint=None,reduce=True),
                Generic_metric(average_coverage,"Cov_all", mask=None,list_ctx_constraint=list_ctx_constraint,reduce=True),
                Generic_metric(sharpness,"Aw_A", mask=None,list_ctx_constraint=list_ctx_constraint,reduce=True,type_var='aleatoric'),
                Generic_metric(sharpness,"Aw_E", mask=None,list_ctx_constraint=list_ctx_constraint,reduce=True,type_var='epistemic')]
dict_perf = evaluate(storing, list_name, list_metrics,verbose=1,cv_list=cv_list)

cv_1 len_train 6553
cv_2 len_train 6553
cv_3 len_train 6554
cv_4 len_train 6554
cv_5 len_train 6554
RF_dUQ  |time_fit : 5.49 time_pred : 12.31
Rmse_all Train 0.072 ± 0.001 | TEST 0.143 ± 0.001
NLL Train -0.941 ± 0.008 | TEST -0.535 ± 0.004
Cov_all Train 1.0 ± 0.0 | TEST 0.993 ± 0.001
Aw_A Train 0.307 ± 0.004 | TEST 0.312 ± 0.003
Aw_E Train 0.492 ± 0.003 | TEST 0.637 ± 0.003

PNN_MCDP  |time_fit : 202.37 time_pred : 7.51
Rmse_all Train 0.049 ± 0.0 | TEST 0.069 ± 0.001
NLL Train -1.528 ± 0.007 | TEST -1.293 ± 0.016
Cov_all Train 0.996 ± 0.0 | TEST 0.963 ± 0.006
Aw_A Train 0.251 ± 0.002 | TEST 0.251 ± 0.002
Aw_E Train 0.15 ± 0.001 | TEST 0.151 ± 0.002

PNN_DE  |time_fit : 1001.02 time_pred : 5.26
Rmse_all Train 0.057 ± 0.0 | TEST 0.067 ± 0.001
NLL Train -1.452 ± 0.004 | TEST -1.33 ± 0.013
Cov_all Train 0.992 ± 0.001 | TEST 0.976 ± 0.004
Aw_A Train 0.284 ± 0.002 | TEST 0.284 ± 0.002
Aw_E Train 0.069 ± 0.002 | TEST 0.076 ± 0.002

EDL  |time_fit : 218.94 time_pred : 0.96
Rmse_all Train 0.048

In [6]:
#Performances for the 4 approaches on Benchmark_public_dataset_Folds5x2_pp with 5-cross val
from abench.benchmark import evaluate, Generic_metric
from src.Benchmark_UQ import rmse,average_coverage,sharpness,Gaussian_NLL

storing =path+'results/Benchmark_public/Benchmark_public_dataset_Folds5x2_pp'
cv_list =  ['cv_1','cv_2','cv_3','cv_4','cv_5']
list_name = ['RF_dUQ','PNN_MCDP','PNN_DE','EDL']
list_metrics = [Generic_metric(rmse,'Rmse_all', mask=None,list_ctx_constraint=None,reduce=True),
                Generic_metric(Gaussian_NLL,"NLL", mask=None,list_ctx_constraint=None,reduce=True),
                Generic_metric(average_coverage,"Cov_all", mask=None,list_ctx_constraint=None,reduce=True),
                Generic_metric(sharpness,"Aw_A", mask=None,list_ctx_constraint=None,reduce=True,type_var='aleatoric'),
                Generic_metric(sharpness,"Aw_E", mask=None,list_ctx_constraint=None,reduce=True,type_var='epistemic')]
dict_perf = evaluate(storing, list_name, list_metrics,verbose=1,cv_list=cv_list)

cv_1 len_train 7654
cv_2 len_train 7654
cv_3 len_train 7654
cv_4 len_train 7655
cv_5 len_train 7655
RF_dUQ  |time_fit : 1.73 time_pred : 2.16
Rmse_all Train 3.276 ± 0.04 | TEST 3.692 ± 0.126
NLL Train 2.614 ± 0.006 | TEST 2.693 ± 0.003
Cov_all Train 0.991 ± 0.001 | TEST 0.98 ± 0.001
Aw_A Train 14.472 ± 0.1 | TEST 14.522 ± 0.108
Aw_E Train 8.173 ± 0.069 | TEST 8.735 ± 0.057

PNN_MCDP  |time_fit : 404.99 time_pred : 9.63
Rmse_all Train 3.552 ± 0.041 | TEST 3.754 ± 0.123
NLL Train 2.579 ± 0.006 | TEST 2.648 ± 0.006
Cov_all Train 0.964 ± 0.001 | TEST 0.949 ± 0.002
Aw_A Train 13.299 ± 0.09 | TEST 13.336 ± 0.09
Aw_E Train 4.406 ± 0.066 | TEST 4.407 ± 0.073

PNN_DE  |time_fit : 1245.34 time_pred : 4.39
Rmse_all Train 2.818 ± 0.075 | TEST 3.449 ± 0.123
NLL Train 2.147 ± 0.027 | TEST 2.552 ± 0.018
Cov_all Train 0.958 ± 0.002 | TEST 0.872 ± 0.006
Aw_A Train 8.718 ± 0.194 | TEST 8.703 ± 0.229
Aw_E Train 2.898 ± 0.179 | TEST 3.102 ± 0.188

EDL  |time_fit : 385.54 time_pred : 1.23
Rmse_all Train 3.

In [7]:
#Performances for the 4 approaches on Benchmark_public_dataset_CASP_protein with 5-cross val
from abench.benchmark import evaluate, Generic_metric
from src.Benchmark_UQ import rmse,average_coverage,sharpness,Gaussian_NLL

storing =path+'results/Benchmark_public/Benchmark_public_dataset_CASP_protein'
cv_list =  ['cv_1','cv_2','cv_3','cv_4','cv_5']
list_name = ['RF_dUQ','PNN_MCDP','PNN_DE','EDL']
list_metrics = [Generic_metric(rmse,'Rmse_all', mask=None,list_ctx_constraint=None,reduce=True),
                Generic_metric(Gaussian_NLL,"NLL", mask=None,list_ctx_constraint=None,reduce=True),
                Generic_metric(average_coverage,"Cov_all", mask=None,list_ctx_constraint=None,reduce=True),
                Generic_metric(sharpness,"Aw_A", mask=None,list_ctx_constraint=None,reduce=True,type_var='aleatoric'),
                Generic_metric(sharpness,"Aw_E", mask=None,list_ctx_constraint=None,reduce=True,type_var='epistemic')]
dict_perf = evaluate(storing, list_name, list_metrics,verbose=1,cv_list=cv_list)

cv_1 len_train 36584
cv_2 len_train 36584
cv_3 len_train 36584
cv_4 len_train 36584
cv_5 len_train 36584
RF_dUQ  |time_fit : 23.45 time_pred : 26.26
Rmse_all Train 2.413 ± 0.004 | TEST 3.606 ± 0.043
NLL Train 2.287 ± 0.004 | TEST 2.503 ± 0.01
Cov_all Train 0.997 ± 0.0 | TEST 0.975 ± 0.002
Aw_A Train 10.9 ± 0.034 | TEST 10.957 ± 0.025
Aw_E Train 9.174 ± 0.037 | TEST 10.814 ± 0.094

PNN_MCDP  |time_fit : 1402.87 time_pred : 49.52
Rmse_all Train 3.571 ± 0.129 | TEST 3.778 ± 0.084
NLL Train 2.28 ± 0.063 | TEST 2.351 ± 0.046
Cov_all Train 0.972 ± 0.001 | TEST 0.957 ± 0.004
Aw_A Train 5671680845.852 ± 11343361667.517 | TEST 10951513.707 ± 21903003.099
Aw_E Train 4.151 ± 0.08 | TEST 4.163 ± 0.07

PNN_DE  |time_fit : 5252.35 time_pred : 21.39
Rmse_all Train 2.884 ± 0.086 | TEST 3.476 ± 0.081
NLL Train 1.766 ± 0.069 | TEST 2.055 ± 0.06
Cov_all Train 0.973 ± 0.002 | TEST 0.917 ± 0.005
Aw_A Train 7717845.928 ± 15342779.291 | TEST 25712505.627 ± 51245862.772
Aw_E Train 3.134 ± 0.121 | TEST 3.408 ±

In [8]:
#Performances for the 4 approaches on Benchmark_public_dataset_year_prediction without cross-val
from abench.benchmark import evaluate, Generic_metric
from src.Benchmark_UQ import rmse,average_coverage,sharpness,Gaussian_NLL

storing =path+'results/Benchmark_public/Benchmark_public_dataset_year_prediction'
cv_list =  ['cv_1']
list_name = ['RF_dUQ','PNN_MCDP','PNN_DE','EDL']
list_ctx_constraint=None
list_metrics = [Generic_metric(rmse,'Rmse_all', mask=None,list_ctx_constraint=None,reduce=True),
                Generic_metric(Gaussian_NLL,"NLL", mask=None,list_ctx_constraint=None,reduce=True),
                Generic_metric(average_coverage,"Cov_all", mask=None,list_ctx_constraint=None,reduce=True),
                Generic_metric(sharpness,"Aw_A", mask=None,list_ctx_constraint=None,reduce=True,type_var='aleatoric'),
                Generic_metric(sharpness,"Aw_E", mask=None,list_ctx_constraint=None,reduce=True,type_var='epistemic')]
dict_perf = evaluate(storing, list_name, list_metrics,verbose=1,cv_list=cv_list)

cv_1 len_train 463716
RF_dUQ  |time_fit : 1923.07 time_pred : 625.35
Rmse_all Train 7.283 ± 0.0 | TEST 9.152 ± 0.0
NLL Train 3.278 ± 0.0 | TEST 3.442 ± 0.0
Cov_all Train 0.992 ± 0.0 | TEST 0.961 ± 0.0
Aw_A Train 29.358 ± 0.0 | TEST 29.462 ± 0.0
Aw_E Train 20.186 ± 0.0 | TEST 21.225 ± 0.0

PNN_MCDP  |time_fit : 24637.9 time_pred : 509.69
Rmse_all Train 8.519 ± 0.0 | TEST 8.753 ± 0.0
NLL Train 3.285 ± 0.0 | TEST 3.305 ± 0.0
Cov_all Train 0.973 ± 0.0 | TEST 0.968 ± 0.0
Aw_A Train 32.886 ± 0.0 | TEST 32.704 ± 0.0
Aw_E Train 5.969 ± 0.0 | TEST 5.956 ± 0.0

PNN_DE  |time_fit : 33615.0 time_pred : 222.65
Rmse_all Train 6.917 ± 0.0 | TEST 8.714 ± 0.0
NLL Train 2.962 ± 0.0 | TEST 3.222 ± 0.0
Cov_all Train 0.967 ± 0.0 | TEST 0.907 ± 0.0
Aw_A Train 20.992 ± 0.0 | TEST 20.983 ± 0.0
Aw_E Train 8.238 ± 0.0 | TEST 9.268 ± 0.0

EDL  |time_fit : 23427.62 time_pred : 63.44
Rmse_all Train 8.687 ± 0.0 | TEST 8.903 ± 0.0
NLL Train 3.283 ± 0.0 | TEST 3.301 ± 0.0
Cov_all Train 0.959 ± 0.0 | TEST 0.955 ± 0.0
